In [7]:
# import statsapi as mlb
# from datetime import datetime
# import time

import pandas as pd
import numpy as np
import plotly

from ..mlb_sqlite.sql_alchemy_mapping import db

db=sql_alchemy_mapping.db

session = sql_alchemy_mapping.session
Base = sql_alchemy_mapping.Base

ValueError: attempted relative import beyond top-level package

In [3]:
from .database import Game

ModuleNotFoundError: No module named '__main__.database'; '__main__' is not a package

In [ ]:
session.execute("""select count(*) from games""").fetchall()

In [6]:
query=(
        """
        select 
            pitch.type_description,
            pitch.play_id,
            play.pitcher_id,
            pitch.startSpeed,
            pitch.endSpeed,
            pitch.zone,
            pitch.spinRate,
            pitch.breakLength
        from 
            pitches pitch 
            inner join plays play 
            on 
            pitch.play_id=play.id
        """
)
default_dict = {99.9:np.nan,0.0:np.nan}
pitch_df=pd.read_sql_query(query,db.db_engine)
pitch_df.replace(default_dict,inplace=True)
pitch_df.dropna(inplace=True)

In [7]:
pitch_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 148151 entries, 0 to 165856
Data columns (total 8 columns):
type_description    148151 non-null object
play_id             148151 non-null object
pitcher_id          148151 non-null float64
startSpeed          148151 non-null float64
endSpeed            148151 non-null float64
zone                148151 non-null float64
spinRate            148151 non-null float64
breakLength         148151 non-null float64
dtypes: float64(6), object(2)
memory usage: 10.2+ MB


In [8]:
pitch_df.type_description.unique()

array(['Slider', 'Four-Seam Fastball', 'Splitter', 'Curveball',
       'Two-Seam Fastball', 'Changeup', 'Sinker', 'Cutter',
       'Knuckle Curve', 'Eephus', 'Forkball'], dtype=object)

In [19]:
# scatter plot of pitches by spin rate and end speed
fig = {
    "data":[
        dict(
            x=pitch_df[pitch_df['type_description'] == i]['spinRate'],
            y=pitch_df[pitch_df['type_description'] == i]['endSpeed'],
            text=pitch_df[pitch_df['type_description'] == i]['pitcher_id'],
            mode='markers',
            marker={
                'size':5,
                'line':{'width':.3,'color':'white'}
            },
            name=i
        ) for i in pitch_df.type_description.unique()
    ],
    "layout":
        dict(
            xaxis={'type': 'log', 'title': 'Spin Rate'},
            yaxis={'type': 'log','title': 'End Speed'},
            autosize=True,
            #margin={'1':40,'b':40,'t':10,'r':10},
            legend={'x':0,'y':1},
            hovermode='closest'
            )
}

In [21]:
# import plotly.io as pio
# pio.show(fig)

In [22]:
import plotly.express as px

In [26]:
df = pd.read_sql('''select * from pitches''',db.db_engine)

In [27]:
df.columns

Index(['id', 'call_code', 'call_description', 'ballColor', 'trailColor',
       'isInPlay', 'isStrike', 'isBall', 'type_code', 'type_description',
       'hasReview', 'count_balls', 'count_strikes', 'startSpeed', 'endSpeed',
       'strikeZoneTop', 'strikeZoneBottom', 'aY', 'aZ', 'pfxX', 'pfxZ', 'pX',
       'pZ', 'vX0', 'vY0', 'vZ0', 'x', 'y', 'x0', 'y0', 'z0', 'aX',
       'breakAngle', 'breakLength', 'breakY', 'spinRate', 'spinDirection',
       'zone', 'typeConfidence', 'plateTime', 'extension', 'index', 'pfxId',
       'pitchId', 'startTime', 'endTime', 'play_id'],
      dtype='object')

In [28]:
df['call_description'].value_counts()

Ball                         54400
Foul                         27348
Called Strike                26330
In play, out(s)              19562
Swinging Strike              19234
In play, no out               6993
In play, run(s)               4183
Ball In Dirt                  3668
Foul Tip                      1418
Swinging Strike (Blocked)     1227
Automatic Ball                 540
Hit By Pitch                   495
Foul Bunt                      367
Missed Bunt                     76
Pitchout                        16
Name: call_description, dtype: int64

In [33]:
df[df['call_description']=='Called Strike']['count_balls'].value_counts()

0    15547
1     5890
2     2962
3     1931
Name: count_balls, dtype: int64

In [36]:
df['spinDirection'].value_counts()

99.9     14790
206.0     1523
204.0     1522
207.0     1508
205.0     1501
         ...  
0.0         40
352.0       39
358.0       33
2.0         33
353.0       29
Name: spinDirection, Length: 361, dtype: int64